In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.models import Model
import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, Reshape, LSTM, BatchNormalization
from keras.models import Sequential
from tensorflow.keras import optimizers
import keras_tuner as kt
from keras_tuner.engine.hyperparameters import HyperParameters
from keras_tuner import RandomSearch, BayesianOptimization
from tensorflow.keras.optimizers.legacy import Adam, Adagrad, Adadelta, SGD,RMSprop
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
img_height = 224
img_width = 224
data_dir = 'data/'

In [ ]:
tr = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    seed=123,
    image_size=(224, 224),
    validation_split=0.05,
    subset='training',
    shuffle = True
)

ts = tf.keras.preprocessing.image_dataset_from_directory(
     data_dir,
     seed=123,
     image_size=(img_height,img_width),
     validation_split=0.05,
     subset="validation",
     shuffle = True
)

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss",patience=7, verbose=1)

class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data,training_data):
        super(MetricsCallback, self).__init__()
        self.validation_data = validation_data
        self.training_data = training_data
        self.true_labels = []
        self.predicted_labels = []
        self.true_labels1 = []
        self.predicted_labels1 = []

    def on_epoch_end(self, epoch, logs=None):
        # Training metrics
        for images, labels in self.training_data:
            self.true_labels1.extend(labels.numpy())
            predictions = model.predict(images, verbose=0)
            self.predicted_labels1.extend(tf.argmax(predictions, axis=1).numpy())
        
        # Calculate metrics
        train_precision = precision_score(self.true_labels1, self.predicted_labels1, average='weighted')
        train_recall = recall_score(self.true_labels1, self.predicted_labels1, average='weighted')
        train_f1 = f1_score(self.true_labels1, self.predicted_labels1, average='weighted')

#         print(f'Epoch {epoch + 1} - Validation Metrics: F1 Score: {val_f1}, Precision: {val_precision}, Recall: {val_recall}')
        # Validation metrics
        
        for images, labels in self.validation_data:
            self.true_labels.extend(labels.numpy())
            predictions = model.predict(images, verbose=0)
            self.predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())
        
        # Calculate metrics
        val_precision = precision_score(self.true_labels, self.predicted_labels, average='weighted')
        val_recall = recall_score(self.true_labels, self.predicted_labels, average='weighted')
        val_f1 = f1_score(self.true_labels, self.predicted_labels, average='weighted')

        print(f'  Train_F1 Score: {train_f1},Val_F1 Score: {val_f1} , Train_Precision: {train_precision} , Val_Precision: {val_precision}, Train_Recall: {train_recall}, Val_Recall: {val_recall}')

# Example usage during training
callback = MetricsCallback(validation_data=ts,training_data=tr)

In [ ]:
def build_model(hp):
    
    effnet = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(img_height,img_width,3))
    output = effnet.layers[-1].output
    output = tf.keras.layers.Flatten()(output)
    output = Reshape((1, -1))(output)
    effnet = Model(effnet.input, output)
    for layer in effnet.layers:
        layer.trainable = False
    
    model1 = Sequential()
    model1.add(effnet)
    
    unit_params = hp.Choice('units', values = [512, 1024, 2048])
    dropout_params = hp.Choice('dropouts', values = [0.2, 0.0])
    optimizer_params = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop','Adadelta', 'Adagrad'])
    learning_rate_params = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
    
    model1.add(LSTM(units=128, return_sequences=False, input_shape=(1, base_model.output_shape[1])))
    model1.add(BatchNormalization())
    model1.add(Dense(units = unit_params, activation='relu', input_dim=(img_height,img_width,3)))
    model1.add(tf.keras.layers.Dropout(0.2))
    model1.add(Dense(7, activation="softmax"))
    
    
    if optimizer_params == 'adam':
        optimizer = Adam(learning_rate=learning_rate_params)
    elif optimizer_params == 'sgd':
        optimizer = SGD(learning_rate=learning_rate_params)
    elif optimizer_params == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adagrad':
        optimizer = Adagrad(learning_rate=learning_rate_params)
    elif optimizer_params == 'Adadelta':
        optimizer = Adadelta(learning_rate=learning_rate_params)
    else:
        raise ValueError(f"Invalid optimizer choice: {optimizer_choice}")

    
    model1.compile(optimizer = optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model1


## Random Search

In [ ]:
tuner = RandomSearch(build_model, objective= kt.Objective("val_accuracy", direction="max"), max_trials = 15 , executions_per_trial=1, directory='logs/Randomsearch/effNet_LSTM', project_name='effNet_LSTM_tuned')
tuner.search(tr, epochs = 5, 
             validation_data=ts, 
             callbacks = [early_stopping,tf.keras.callbacks.TensorBoard("logs/Randomsearch/effNet_LSTM")])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_hps.values

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(tr, epochs=30, validation_data = ts, callbacks = [early_stopping, callback])

In [ ]:
class_names = ts.class_names

# Get true labels and model predictions
true_labels = []
predicted_labels = []

for images, labels in ts:
    true_labels.extend(labels.numpy())
    predictions = model.predict(images)
    predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())

# Calculate metrics
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
    
    
# # Find misclassified samples
# misclassified_indices = [i for i, (true, pred) in enumerate(zip(true_labels, predicted_labels)) if true != pred]
# misclassified_samples = [(class_names[true], class_names[pred]) for true, pred in zip(true_labels, predicted_labels) if true != pred]

# # Print misclassifications
# for sample in misclassified_samples:
#     print(f"True: {sample[0]}, Predicted: {sample[1]}")

# # Calculate overall accuracy
# accuracy = 1 - (len(misclassified_indices) / len(true_labels))
# print(f"Overall accuracy: {accuracy}")

## Bayesian Optimization

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss",patience=7, verbose=1)

tuner = RandomSearch(build_model, objective= kt.Objective("val_accuracy", direction="max"), max_trials = 30 , executions_per_trial=1, directory='logs/Bayesiansearch/effNet_LSTM', project_name='effNet_tuned_LSTM')
tuner.search(tr, epochs = 5, 
             validation_data=ts, 
             callbacks = [early_stopping,tf.keras.callbacks.TensorBoard("logs/Bayesiansearch/effNet_LSTM")])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
best_hps.values

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(tr, epochs=30, validation_data = ts, callbacks = [early_stopping, callback])

In [ ]:
class_names = ts.class_names

# Get true labels and model predictions
true_labels = []
predicted_labels = []

for images, labels in ts:
    true_labels.extend(labels.numpy())
    predictions = model.predict(images)
    predicted_labels.extend(tf.argmax(predictions, axis=1).numpy())

# Calculate metrics
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
    
    
# # Find misclassified samples
# misclassified_indices = [i for i, (true, pred) in enumerate(zip(true_labels, predicted_labels)) if true != pred]
# misclassified_samples = [(class_names[true], class_names[pred]) for true, pred in zip(true_labels, predicted_labels) if true != pred]

# # Print misclassifications
# for sample in misclassified_samples:
#     print(f"True: {sample[0]}, Predicted: {sample[1]}")

# # Calculate overall accuracy
# accuracy = 1 - (len(misclassified_indices) / len(true_labels))
# print(f"Overall accuracy: {accuracy}")